In [11]:
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Input, Dense

In [4]:
housing_data = pd.read_csv('sample_data/california_housing_train.csv')

In [5]:
housing_data.describe().T

,count,mean,std,min,25%,50%,75%,max
longitude,17000.0,-119.562108,2.005166,-124.3500,-121.790000,-118.4900,-118.000,-114.3100
latitude,17000.0,35.625225,2.137340,32.5400,33.930000,34.2500,37.720,41.9500
housing_median_age,17000.0,28.589353,12.586937,1.0000,18.000000,29.0000,37.000,52.0000
total_rooms,17000.0,2643.664412,2179.947071,2.0000,1462.000000,2127.0000,3151.250,37937.0000
total_bedrooms,17000.0,539.410824,421.499452,1.0000,297.000000,434.0000,648.250,6445.0000
population,17000.0,1429.573941,1147.852959,3.0000,790.000000,1167.0000,1721.000,35682.0000
households,17000.0,501.221941,384.520841,1.0000,282.000000,409.0000,605.250,6082.0000
median_income,17000.0,3.883578,1.908157,0.4999,2.566375,3.5446,4.767,15.0001
median_house_value,17000.0,207300.912353,115983.764387,14999.0000,119400.000000,180400.0000,265000.000,500001.0000


In [6]:
housing_data.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value'],
      dtype='object')

In [7]:
housing_data.dtypes

longitude             float64
latitude              float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
median_house_value    float64
dtype: object

In [15]:
num_columns = housing_data.shape[1]
num_outputs = 1

In [13]:
X = housing_data.drop(columns= ['median_house_value'])
y = housing_data.median_house_value

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [16]:
model = Sequential([
    Input(num_columns),
    Dense(32, activation = 'relu'),
    Dense(64, activation = 'relu'),
    Dense(32, activation = 'relu'),
    Dense(num_outputs, activation = None),
])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                320       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4545 (17.75 KB)
Trainable params: 4545 (17.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
model.weights[0].numpy()

array([[-1.89642712e-01, -1.24507427e-01,  1.31441623e-01,
         2.05748886e-01, -2.64535129e-01, -3.56989622e-01,
         2.30610758e-01,  3.18427771e-01, -3.32170904e-01,
         2.35913664e-01, -9.04455185e-02,  3.81119281e-01,
        -1.29391789e-01, -2.02705696e-01, -5.22122979e-02,
         2.89600819e-01,  2.57931143e-01, -4.02474999e-02,
         3.28643709e-01,  2.02137858e-01, -3.27524781e-01,
        -3.76088470e-01,  3.01258415e-01,  3.29166025e-01,
         3.25138837e-01,  8.13384056e-02,  9.54127014e-02,
        -3.29170823e-02,  3.49492520e-01,  1.69381529e-01,
        -1.24565333e-01,  3.44023854e-01],
       [ 1.34113878e-01,  3.56545448e-02,  7.08308220e-02,
         1.78884119e-01, -2.73008525e-01,  5.70663810e-02,
        -1.61055446e-01,  2.21767724e-02, -3.29929233e-01,
        -3.46284866e-01,  2.65579224e-02, -3.67003977e-01,
         2.37171918e-01,  1.32415563e-01,  3.64244908e-01,
         3.29630673e-02,  2.26749927e-01, -2.01365158e-01,
         3.66